In [2]:
import json
from pykospacing import Spacing
from konlpy.tag import Okt 
import os
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
from gensim.models import KeyedVectors
import pickle
import re

tokenized_data = []
tokenized_data_valid = []
file_list = []
y_train = []
y_valid = []
max_len = 500
file_len = 0
file_len_valid = 0
embedding_dim = 100
category = ["농림.축산지원", "도시개발", "산업진흥", "상.하수도관리", "인.허가", "일반행정", "주민복지", "주민생활지원", "주민자치", "지역문화", "지연환경.산림", "회계.예산"]
train_paths = ["./Training/train/01.라벨링데이터(Json)", "./Training/train2"]
valid_paths = ["./Validation/valid/01.라벨링데이터(Json)"]
filePath_x = './X_train.txt'
filePath_y = './Y_train.txt'
filePath_valx = './X_valid.txt'
filePath_valy = './Y_valid.txt'

hangul = re.compile('[^ ㄱ-ㅣ가-힣+]')

def get_tokenized(file_path):
    label = []
    with open(file_path, 'r', encoding='UTF8') as f:
        json_data = json.load(f)

    org_sentence = ""
    
    for i in range(len(json_data['annotations'])):
        temp = str(json_data['annotations'][i]['annotation.text'])
        if temp == ' ':
            continue
        org_sentence += temp
    lab = str(json_data['images'][0]['image.category'])
    if lab in category:
        label.append(str(json_data['images'][0]['image.category']))
    else:
        if lab == '농립.축산지원':
            label.append('농림.축산지원')
        elif lab == '상수도관리':
            label.append('상.하수도관리')
        
    spacing = Spacing()
    okt = Okt()
    prep_sentence = hangul.sub('', org_sentence)
    space_sentence = spacing(prep_sentence)
    tok_sentence = okt.nouns(space_sentence)
    return tok_sentence, label

In [ ]:
#ver = 0
#finished = ['농림.축산지원', '도시개발', '산업진흥', '상.하수도관리', '인.허가', '일반행정', '주민복지', '주민생활지원']
file_max = 100
for dir_path in train_paths:
    #ver += 1
    for cate in category:
        file_len = 0
        cate_path = os.path.join(dir_path, cate)
        for (root, directories, files) in os.walk(cate_path):            
            print("Saving directory : ", root)
            if file_len == file_max:
                break
            for file in files:
                if file_len == file_max:
                    break
                file_path = os.path.join(root, file)
                try:
                    tok_sentence, label = get_tokenized(file_path)
                    tokenized_data.append(tok_sentence)
                    #tmp.append(tok_sentence)
                    y_train.append(label)
                    #tmp_y.append(label)
                    file_len += 1
                except:
                    continue
            
with open('tokenized_data.txt', 'wb') as lf:
    pickle.dump(tokenized_data, lf)
with open('y_train_org.txt', 'wb') as lf:
    pickle.dump(y_train, lf)
    
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_data)
vocab_size = len(tokenizer.word_index)
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(tokenized_data)
X_train = tokenizer.texts_to_sequences(tokenized_data)
X_train = pad_sequences(X_train, maxlen=max_len)

tokenizer_y = Tokenizer()
tokenizer_y.fit_on_texts(y_train)
Y_train = tokenizer_y.texts_to_sequences(y_train)
Y_train = to_categorical(Y_train)
#print(Y_train.shape) # (100, 2)
#print(X_train.shape) # (100, 260)

# Save X_train, Y_train
with open(filePath_x, 'wb') as lf:
    pickle.dump(X_train, lf)

with open(filePath_y, 'wb') as lf:
    pickle.dump(Y_train, lf)
    
  
tokenized_data_valid = []
y_valid = []
for dir_path in valid_paths:
    for cate in category:
    #    tmp = []
    #    tmp_y = []
        file_len_valid = 0
        dir_path2 = os.path.join(dir_path, cate)
        for (root, directories, files) in os.walk(dir_path2):  
            if file_len_valid == file_max:
                break          
            print("Saving directory : ", root)            
            for file in files:
                if file_len_valid == file_max:
                    break  
                file_path = os.path.join(root, file)
                try:
                    tok_sentence, label = get_tokenized(file_path)
                    tokenized_data_valid.append(tok_sentence)
                    y_valid.append(label)
                    file_len_valid += 1
                except:
                    continue
                
X_valid = tokenizer.texts_to_sequences(tokenized_data_valid)
X_valid = pad_sequences(X_valid, maxlen=max_len)

Y_valid = tokenizer_y.texts_to_sequences(y_valid)
Y_valid = to_categorical(Y_valid)

with open('tokenized_data_valid.txt', 'wb') as lf:
    pickle.dump(tokenized_data_valid, lf)
    
with open('y_valid_org.txt', 'wb') as lf:
    pickle.dump(y_valid, lf)

# Save X_valid, Y_valid
with open(filePath_valx, 'wb') as lf:
    pickle.dump(X_valid, lf)

with open(filePath_valy, 'wb') as lf:
    pickle.dump(Y_valid, lf)